2. Multiple Input Series: 둘 이상의 병렬적 입력 시계열과 하나의 출력 시계열로 구성
   (출력 시계열은 입력 시계열과 연관되어 있음)

In [1]:
from tensorflow.python.client import device_lib
import keras.backend.tensorflow_backend as K
#print(device_lib.list_local_devices())

import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
np.random.seed(3)

Using TensorFlow backend.


In [2]:
# split a multivariate sequence into samples
def split_sequences_multi(sequences, n_steps):
    X, y = list(), list()
    len_ = len(sequences)
    for i in range(len_):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1] #마지막열이 y이기떄문에 -1
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#예제 데이터 셋 정의
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [4]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences_multi(dataset, n_steps)

In [5]:
###### flatten input: 여러개의 백터를 한 백터로 나타내준다. ######
n_input = X.shape[1] * X.shape[2] #백터의 길이
X = X.reshape((X.shape[0], n_input))

In [6]:
# define model
with K.tf.device('/cpu:0'): #'/cpu:0'
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_input))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=2000, verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
# 예측
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[206.1564]]
